In [1]:
import torch
import os
# print(os.getcwd())
import random
import numpy as np
import albumentations as A
import pandas as pd
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from Cyto_dataset import *
from torchvision import models
import torch.nn as nn
from utils import *
from classifier_utils import *
import torchvision
from torchvision import models
import time

from classifier_utils_sw import train,test
#import argparse

seed=20
seed_everything(seed)


# parser = argparse.ArgumentParser(description='Arg')
# parser.add_argument('--trial', type=str)
# parser.add_argument('--image_size', type=int, default=456)
# parser.add_argument('--batch_size', type=int, default=8)
# parser.add_argument('--n_epochs', type=int, default=16)
# parser.add_argument('--lr', type=float, default=1e-5)

# args = parser.parse_args()


n_cls = 2
    
ben_prepro = False
# image_size = (args.image_size, args.image_size)
# batch_size = args.batch_size
# n_epochs = args.n_epochs
# lr = args.lr
Full_setting= True
# lr = 2.5e-5
image_size = 320
batch_size = 32
n_epochs = 15
lr = 1e-6
trial ='image_size_320_cos_full'
 

#scale_te = A.Resize(256, 256)
scale = A.Resize(image_size, image_size)
#rcrop = A.RandomCrop(width=256, height=256)
rotate = A.Rotate(limit=15)
H_flip = A.HorizontalFlip(p=0.5)
V_flip = A.VerticalFlip(p=0.5)
cutout = A.Cutout(num_holes=8, max_h_size=8, max_w_size=8, p=0.5)
elastic = A.ElasticTransform(p=0.5)
RBC = A.RandomBrightnessContrast(p=0.5)
blur = A.Blur(blur_limit=24, p=0.5)
HSV = A.HueSaturationValue(p=0.5)
ssr = A.ShiftScaleRotate(rotate_limit=30, p=0.5)

normalize = A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
to_tensor = ToTensorV2()

train_transform = A.Compose(
        [
            scale,
            #rcrop,
            #rotate,
            H_flip,          
            V_flip,
            ssr,
            elastic,
            #blur,
            cutout,
            #RBC,

            normalize,
            to_tensor
        ])

test_transform = A.Compose(
        [
            scale,
            #scale_te,
            normalize,
            to_tensor
        ])


train_list, val_list = train_val_split('/home/ec2-user/dataset/b-trac-cyto/final', train_ratio=0.9)
#train_fold, val_fold = fold5_split('/home/ec2-user/dataset/b-trac-cyto/final')



train_set = Cyto_train_dataset(train_list, transform=train_transform)
val_set = Cyto_train_dataset(val_list, transform=test_transform)

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=4)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=4)



print(f'num_train : {len(train_loader)}, num_val : {len(val_loader)}')

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
model.fc = nn.Linear(num_ftrs, 2)
#model.load_state_dict(torch.load('./weights/pretrain_model.pth'))
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
model.to(device)

weight_decay = 1e-5
criterion = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay )
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.00005, 
                                                steps_per_epoch=2, epochs=15,anneal_strategy='cos')
print(f'n_epoch:{n_epochs}, lr:{lr}, batch_size:{batch_size}')



createFolder('./check_point/'+trial)

print(trial)

best_model, best_epoch = train(model, n_cls, n_epochs, train_loader, val_loader, criterion, optimizer, scheduler, trial, device)

#torch.save(best_model.state_dict(), './check_point/final_model.pth')


print("done")



seed : 20
neg_listdir :  4000 pos_listdir :  4000
len train :  7200 len val :  800
num_train : 225, num_val : 25


/home/ec2-user/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:691: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


n_epoch:15, lr:1e-06, batch_size:32
image_size_320_cos_full
image_size_320_cos_full
n_epoch:0
[epoch_1, batch_100] loss: 0.6167233437299728, acc: 0.7053125, f1: 0.7079821527222225
[epoch_1, batch_200] loss: 0.4776185804605484, acc: 0.8359375, f1: 0.8260694474621424
perf_counter : 136.27989125601016
process_time : 54.304402895
train_loss: 0.5313847617308298 train_acc 0.7797222222222222 train_f1 0.7747798920761148 train_avg 0.7790784506953825
valid_loss: 0.3481889677047729 valid_acc: 0.8775 valid_f1: 0.8703703703703702 valid_avg: 0.8778529606385411
lr :  3.826891219729119e-06
current epoch: 0
0 save weight
image_size_320_cos_full
n_epoch:1
[epoch_2, batch_100] loss: 0.3281785722076893, acc: 0.88125, f1: 0.8769720895566023
[epoch_2, batch_200] loss: 0.2751482970267534, acc: 0.8934375, f1: 0.8882564997431391
perf_counter : 138.32673852000153
process_time : 54.38966916300001
train_loss: 0.2969418840607007 train_acc 0.8879166666666667 train_f1 0.8845328373157819 train_avg 0.8877984248127434


done
